<a href="https://colab.research.google.com/github/knightkjt/FSRCNN_yippp_pub/blob/master/fsrcnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

import os
import sys
from google.colab import files
from google.colab import drive

# mount google drive
drive.mount('/content/gdrive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [6]:
!ls gdrive/My\ Drive\/Colab\ Notebooks/FSRCNN/FSRCNN-master

bmp	      dataset	 logs	  main.py  model.py   Report.md
butterfly.py  logger.py  loss.py  misc.py  README.md  solver.py


In [0]:
cp -rf gdrive/My\ Drive\/Colab\ Notebooks/FSRCNN/FSRCNN-master/* .


In [17]:
def getLocalFiles(path_): 
    _files = os.listdir(path_)
    print (_files)
    if len(_files) >0: 
        for l in _files:
            ext = l.lower().split('.')[-1]
            k = path_ + l
            #print (ext)
            if (not os.path.isdir(k) and not ext == 'ipynb'):
                #import pdb;pdb.set_trace()
                v = open(k,'rb').read()
                open(k,'wb').write(v) 
!ls
getLocalFiles('./')   
getLocalFiles('./dataset/')
        


adc.json      dataset	 logs	  misc.py    Report.md
bmp	      gdrive	 loss.py  model.py   sample_data
butterfly.py  logger.py  main.py  README.md  solver.py
['.config', 'loss.py', 'misc.py', 'butterfly.py', 'main.py', 'logs', 'adc.json', 'model.py', 'gdrive', 'dataset', 'bmp', 'solver.py', 'Report.md', 'logger.py', 'README.md', 'sample_data']
['generate_test.m', 'test', 'img_cut.py', 'train', 'modcrop.m', 'generate_train.m', 'store2hdf5.m', 'PairRandomCrop.py', 'data.py', 'dataset.py', 'data_aug.m']
['.config', 'loss.py', 'misc.py', 'butterfly.py', 'main.py', 'logs', 'adc.json', 'model.py', 'gdrive', 'dataset', 'bmp', 'solver.py', 'Report.md', 'logger.py', 'README.md', 'sample_data']


In [0]:
#!touch dataset/__init__.py
from dataset.data import *


In [27]:

!python main.py --train_set=train/91-images

===> Loading datasets
Traceback (most recent call last):
  File "main.py", line 38, in <module>
    main()
  File "main.py", line 22, in main
    train_set = get_h5_set(args.train_set)
  File "/content/dataset/data.py", line 12, in get_h5_set
    return LoadH5(train_dir)
  File "/content/dataset/dataset.py", line 40, in __init__
    with h5py.File(image_h5, 'r') as hf:
  File "/usr/local/lib/python2.7/dist-packages/h5py/_hl/files.py", line 312, in __init__
    fid = make_fid(name, mode, userblock_size, fapl, swmr=swmr)
  File "/usr/local/lib/python2.7/dist-packages/h5py/_hl/files.py", line 142, in make_fid
    fid = h5f.open(name, flags, fapl=fapl)
  File "h5py/_objects.pyx", line 54, in h5py._objects.with_phil.wrapper
  File "h5py/_objects.pyx", line 55, in h5py._objects.with_phil.wrapper
  File "h5py/h5f.pyx", line 78, in h5py.h5f.open
IOError: Unable to open file (file read failed: time = Sun Aug  4 11:30:27 2019
, filename = './dataset/train/91-images', file descriptor = 3, errno =

In [0]:
#------------------------------------------------------------------------------
# The famous data set:cats vs dogs is used in this example. The data set contains
# 12500 dog pictures and 12500 cat pictures. All the images are shuffled randomly
# and 20000 images are used to train, 5000 images are used to test. The images
# can be resized to different sizes but the size of the .hdf5 file differs very
# far depending on the size of the images. The file is 1.14G when the size of the 
# images is (128,128) and 4.57G for (256,256), 18.3G for (512,512).
#------------------------------------------------------------------------------

########################## first part: prepare data ###########################
from random import shuffle
import glob

shuffle_data = True  # shuffle the addresses

hdf5_path = './dataset/train/91.h5'  # file path for the created .hdf5 file

train_path = './dataset/train/91-images/*.bmp' # the original data path

# get all the image paths 
addrs = glob.glob(train_path)

# label the data as 0=cat, 1=dog
#import pdb; pdb.set_trace()
#labels = [0 if 'cat' in addr else 1 for addr in addrs] 
labels = [1 if 't' in addr else 1 for addr in addrs]  

# shuffle data
if shuffle_data:
    c = list(zip(addrs, labels)) # use zip() to bind the images and labels together
    shuffle(c)
 
    (addrs, labels) = zip(*c)  # *c is used to separate all the tuples in the list c,  
                               # "addrs" then contains all the shuffled paths and 
                               # "labels" contains all the shuffled labels.
                               
# Divide the data into 80% for train and 20% for test
train_addrs = addrs[0:int(0.8*len(addrs))]
train_labels = labels[0:int(0.8*len(labels))]

test_addrs = addrs[int(0.8*len(addrs)):]
test_labels = labels[int(0.8*len(labels)):]

##################### second part: create the h5py object #####################
import numpy as np
import h5py

train_shape = (len(train_addrs), 128, 128, 3)
test_shape = (len(test_addrs), 128, 128, 3)

# open a hdf5 file and create earrays 
f = h5py.File(hdf5_path, mode='w')

# PIL.Image: the pixels range is 0-255,dtype is uint.
# matplotlib: the pixels range is 0-1,dtype is float.
f.create_dataset("train_img", train_shape, np.uint8)
f.create_dataset("test_img", test_shape, np.uint8)  

# the ".create_dataset" object is like a dictionary, the "train_labels" is the key. 
f.create_dataset("train_labels", (len(train_addrs),), np.uint8)
f["train_labels"][...] = train_labels

f.create_dataset("test_labels", (len(test_addrs),), np.uint8)
f["test_labels"][...] = test_labels

######################## third part: write the images #########################
import cv2

# loop over train paths
for i in range(len(train_addrs)):
  
    if i % 1000 == 0 and i > 1:
        print ('Train data: {}/{}'.format(i, len(train_addrs)) )

    addr = train_addrs[i]
    img = cv2.imread(addr)
    img = cv2.resize(img, (128, 128), interpolation=cv2.INTER_CUBIC)# resize to (128,128)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # cv2 load images as BGR, convert it to RGB
    f["train_img"][i, ...] = img[None] 

# loop over test paths
for i in range(len(test_addrs)):

    if i % 1000 == 0 and i > 1:
        print ('Test data: {}/{}'.format(i, len(test_addrs)) )

    addr = test_addrs[i]
    img = cv2.imread(addr)
    img = cv2.resize(img, (128, 128), interpolation=cv2.INTER_CUBIC)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    f["test_img"][i, ...] = img[None]

f.close()